## 1. Preprocessing

### 1.1 Import and Load Data

In [10]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### 1.2 Clean Table

In [12]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_2 = application_df.drop(['EIN', 'NAME', 'STATUS'], axis=1)
application_df_2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [13]:
# Determine the number of unique values in each column.
application_df_2.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

### 1.3 Clean Application Type Variable

In [14]:
# Look at APPLICATION_TYPE value counts for binning
application_df_2['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T8', 'T7', 'T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df_2['APPLICATION_TYPE'] = application_df_2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_2['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

### 1.4 Clean Classification Variable

In [16]:
# Look at CLASSIFICATION value counts for binning
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [17]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df_2['CLASSIFICATION'].value_counts()
classification_counts[classification_counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [18]:
classification_counts.index

Index(['C1000', 'C2000', 'C1200', 'C3000', 'C2100', 'C7000', 'C1700', 'C4000',
       'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230',
       'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800',
       'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210',
       'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0',
       'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600',
       'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700',
       'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732',
       'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150'],
      dtype='object')

In [19]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for classification in classification_counts.index:
  if classification not in ['C1000', 'C2000', 'C1200', 'C3000', 'C2100']:
    classifications_to_replace.append(classification)

classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [20]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df_2['CLASSIFICATION'] = application_df_2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

### 1.5 Convert Categorical Variables

In [21]:
application_df_2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [22]:
application_df_2['SPECIAL_CONSIDERATIONS'].unique()

array(['N', 'Y'], dtype=object)

In [23]:
application_df_2['SPECIAL_CONSIDERATIONS'] = np.where(application_df_2['SPECIAL_CONSIDERATIONS']=='Y', 1, 0)
application_df_2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,0,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,0,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,0,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,0,142590,1


In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_3 = pd.get_dummies(application_df_2, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT'])
application_df_3.head()

,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0,5000,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,5000,0,0,0,0,0,1,0,1,...,0,1,0,0,0,0,0,0,0,0
3,0,6692,1,0,0,1,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
4,0,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [25]:
application_df_3['IS_SUCCESSFUL'].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

### 1.6 Split the Data

In [26]:
# Split our preprocessed data into our features and target arrays
y = application_df_3.IS_SUCCESSFUL.values
X = application_df_3.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

### 1.7 Scale the Data

In [27]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## 2. Compile, Train and Evaluate the Model (Optimization 1 - Increase Number of Neurons)

### 2.1 Define the Model

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=160, activation="relu", input_dim=38))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 160)               6240      
                                                                 
 dense_4 (Dense)             (None, 60)                9660      
                                                                 
 dense_5 (Dense)             (None, 1)                 61        
                                                                 
Total params: 15,961
Trainable params: 15,961
Non-trainable params: 0
_________________________________________________________________


### 2.2 Compile the Model

In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### 2.3 Train the Model

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5698 - accuracy: 0.7236
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5574 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5544 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5522 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5498 - accuracy: 0.7338
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5494 - accuracy: 0.7345
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7349
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accura

### 2.4 Evaluate the Model

In [31]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5728 - accuracy: 0.7263 - 356ms/epoch - 1ms/step
Loss: 0.5728253126144409, Accuracy: 0.7262973785400391


## 3. Compile, Train and Evaluate the Model (Optimization 2 - Increase Epochs and Additional Hidden Layer)

### 3.1 Define the Model

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=38))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3120      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 15)                465       
                                                                 
 dense_9 (Dense)             (None, 1)                 16        
                                                                 
Total params: 6,031
Trainable params: 6,031
Non-trainable params: 0
_________________________________________________________________


### 3.2 Compile Model

In [33]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### 3.3 Train the Model

In [34]:
# Train the model
fit_model_2 = nn_2.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 1ms/step - loss: 0.5751 - accuracy: 0.7221
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5570 - accuracy: 0.7304
Epoch 3/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5539 - accuracy: 0.7325
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7336
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7352
Epoch 6/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7341
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7347
Epoch 8/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5493 - accuracy: 0.7341
Epoch 9/150
804/804 [==============================] - 4s 4ms/step - loss: 0.5484 - accuracy: 0.7349
Epoch 10/150
804/804 [==============================] - 3s 4ms/step - loss: 0.5483 - accura

### 3.4 Evaluate the Model

In [35]:
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5777 - accuracy: 0.7248 - 398ms/epoch - 1ms/step
Loss: 0.5777024030685425, Accuracy: 0.724781334400177


## 4. Compile, Train and Evaluate the Model (Optimization 3 - Auto Optimizer)

### 4.1 Create Function

In [36]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_3 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_3.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=38))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_3

### 4.2 Run the kerastuner search for best hyperparameters

In [37]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2)

In [39]:
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 508 Complete [00h 03m 23s]
val_accuracy: 0.7267638444900513

Best val_accuracy So Far: 0.7274635434150696
Total elapsed time: 03h 11m 05s


### 4.3 Best Model Hyperparameters

In [40]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'num_layers': 5,
 'units_0': 7,
 'units_1': 3,
 'units_2': 1,
 'units_3': 3,
 'units_4': 1,
 'units_5': 7,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 34,
 'tuner/bracket': 4,
 'tuner/round': 4,
 'tuner/trial_id': '0142'}

In [41]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7275 - 452ms/epoch - 2ms/step
Loss: 0.5601452589035034, Accuracy: 0.7274635434150696


In [42]:
best_model.save('AlphabetSoupCharity_Optimization.h5')